<a href="https://colab.research.google.com/github/mecfoliveira86/DH/blob/master/Lab_de_Pron%C3%BAncia_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
"""
PROJETO: Laboratório de Pronúncia Multilíngue
OBJETIVO: Prática fonética em Inglês, Alemão, Mandarim, Espanhol e Francês.
AUTOR: Felipe Oliveira (https://github.com/mecfoliveira86)
REFERÊNCIA: Inspirado no projeto da DIO (https://colab.research.google.com/...)

REQUISITOS:
- Chave de API do Gemini (Obtenha em: https://aistudio.google.com/app/apikey)
- Ambiente Google Colab (para suporte a áudio via navegador)

DESCRIÇÃO:
O sistema utiliza Whisper (OpenAI) para transcrição, Gemini (Google) para análise
pedagógica e gTTS para síntese de voz nativa.
"""


In [38]:
#adicionando os recursos externos necessários para rodar o código

!pip install -q -U google-generativeai git+https://github.com/openai/whisper.git gTTS

import os, time, re, whisper, google.generativeai as genai
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# --- CONFIGURAÇÃO ---
# Coloque sua chave aqui
os.environ["GOOGLE_API_KEY"] = "INSIRA AQUI A SUA CHAVE DE API"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Inicializa modelos
whisper_model = whisper.load_model("small")
model_professor = genai.GenerativeModel(
    'models/gemini-2.5-flash',
    system_instruction="Você é um professor de idiomas. Responda no formato FEEDBACK: (texto) CORRETO: (palavra)."
)

# Capturar o áudio por meio do dispositivo de interação do usuário
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_audio(sec=5):
    display(Javascript(RECORD))
    js_result = output.eval_js('record(%s)' % (sec * 1000))
    audio = b64decode(js_result.split(',')[1])
    file_name = 'audio_input.wav'
    with open(file_name, 'wb') as f: f.write(audio)
    return file_name

def tocar_audio(texto, lang='pt'):
    texto_limpo = re.sub(r'[*_#]', '', texto)
    tts = gTTS(text=texto_limpo, lang=lang)
    filename = f"audio_resp.wav"
    tts.save(filename)
    display(Audio(filename, autoplay=True))

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [39]:
# --- EXECUÇÃO DO MENU E PRÁTICA ---
mapa_idiomas = {
    "inglês": "en", "alemão": "de", "francês": "fr", "espanhol": "es", "mandarim": "zh-cn"
}

print("\n--- PASSO 1: Escolha o seu Idioma ---")
print("\n--- Você pode escolher entre Inglês, Mandarim, Espanhol, Francês ou Alemão")
audio_lang = record_audio(sec=3)
res = whisper_model.transcribe(audio_lang, language='pt', fp16=False, initial_prompt="Inglês, Alemão, Francês, Espanhol, Mandarim")
texto_ouvido = res["text"].lower().strip()

# Lógica de seleção
lang_selecionado = next((id for id in mapa_idiomas if id in texto_ouvido), "inglês")
lang_code = mapa_idiomas[lang_selecionado]

print(f"✅ IDIOMA: {lang_selecionado.upper()}")
tocar_audio(f"Vamos praticar {lang_selecionado}. Pode falar agora.")
time.sleep(3.5)

print(f"\n--- PASSO 2: Fale em {lang_selecionado.capitalize()} ---")
audio_pronuncia = record_audio(sec=5)
transcricao = whisper_model.transcribe(audio_pronuncia, fp16=False)["text"]
print(f"Ouvi: '{transcricao}'")

# Feedback Gemini
response = model_professor.generate_content(f"Praticando {lang_selecionado}, disse: '{transcricao}'. Avalie.").text

# Processa e toca
try:
    feedback = response.split("FEEDBACK:")[1].split("CORRETO:")[0].strip()
    correto = response.split("CORRETO:")[1].strip()
except:
    feedback, correto = response, ""

print(f"\n💡 {feedback}")
tocar_audio(feedback)

if correto:
    time.sleep(len(feedback)/15 + 2)
    tocar_audio(f"A pronúncia correta é:", lang='pt')
    time.sleep(2)
    tocar_audio(correto, lang=lang_code)


--- PASSO 1: Escolha o seu Idioma ---

---Você pode escolher entre Inglês, Mandarim, Espanhol, Francês ou Alemão


<IPython.core.display.Javascript object>

✅ IDIOMA: ALEMÃO



--- PASSO 2: Fale em Alemão ---


<IPython.core.display.Javascript object>

Ouvi: ' change... and...GE. Transcribed byаб膩'

💡 Excelente tentativa! Parece que você está tentando dizer 'Obrigado' em alemão. A parte 'change' é um bom esforço para o início, mas o som 'ch' em alemão (no caso de 'Danke') é um som suave, feito na parte superior da boca, quase como um sussurro, e não o som mais duro de 'ch' em 'change' (inglês). A vogal 'a' em 'Danke' é mais aberta, como o 'a' em 'pai' em português. A parte 'GE' está muito boa para o final da palavra! Continue praticando!
